### Importamos lo necesario

In [1]:
import requests
import io
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader
import pandas as pd
import utils_araucania

## Leemos el último dato

In [2]:
araucania = pd.read_csv("output/araucania.csv")
araucania.tail()

,Fecha,Angol,Carahue,Chol-Chol,Collipulli,Cunco,Curacautín,Curarrehue,Ercilla,Freire,...,Purén,Renaico,Saavedra,Temuco,Teodoro Schmidt,Toltén,Traiguén,Victoria,Vilcún,Villarrica
85,10 de Junio,206,68,22,34,40,9,4,16,85,...,8,26,92,849,10,16,19,123,95,34
86,11 de Junio,214,68,22,34,45,10,4,16,87,...,8,26,92,852,10,16,19,125,98,37
87,12 de Junio,224,68,24,34,44,10,4,16,91,...,8,26,92,852,10,16,21,133,98,40
88,13 de Junio,242,68,24,36,47,11,4,18,92,...,8,26,95,861,10,16,26,138,100,43
89,14 de Junio,248,68,24,37,47,11,4,18,94,...,8,27,95,863,10,16,26,139,100,47


In [3]:
ultima_fecha = araucania.tail(1)["Fecha"].values.tolist()
ultima_fecha = ultima_fecha[0]
ultima_fecha

'14 de Junio'

## Capturamos todos los informes

In [4]:
URL = 'https://seremi9.redsalud.gob.cl/accion-sanitaria-3/registros-covid19/'
page = requests.get(URL) #Obtenemos la respuesta del servidor

In [5]:
soup      = BeautifulSoup(page.content, 'html.parser')  #Transformamos a un objeto soup
job_elems = soup.find_all('ol')                    # Buscamos todos los elementos "ol"

for job_elem in job_elems:                         # Recorremos
    enlaces = job_elem.find_all('a')               # Buscamos los enlaces de ahí

In [6]:
reportes_diarios = []
falta_informe    = False
base_url         = "https://seremi9.redsalud.gob.cl"

for enlace in enlaces:                            # Recorremos los enlaces
    fecha  = enlace.text                          # obtenemos el texto
    enlace = enlace['href']                       # obtenemos el enlace
    url    = base_url + enlace
    
    
    if falta_informe:
        dato   = {'fecha': fecha, 'url':url}
        reportes_diarios.append(dato)
    
    if fecha == ultima_fecha:
        falta_informe = True

In [7]:
reportes_diarios

[{'fecha': '15 de Junio',
  'url': 'https://seremi9.redsalud.gob.cl/wrdprss_minsal/wp-content/uploads/2020/06/COMUNICADO-DE-PRENSA-15.05.pdf'}]

## Traemos los informes faltantes

In [8]:
if len(reportes_diarios)>0:
    ciudades = utils_araucania.get_citys(reportes_diarios[0]['fecha'], reportes_diarios[0]['url'])
else:
    print ("Todo Ok...")

Obteniendo información del: 15 de Junio
['                                    ', 'COMUNICADO DE PRENSA', ' ', '2.', '728', ' ', 'son', ' ', 'los casos confirmados de', ' ', 'COVID', '-', ' ', '19 en La Araucanía', '.', ' ', 'Lunes 15', ' ', 'de ', 'junio', ' ', '2020. ', 'Durante ', 'el ', 'día ayer fueron', ' ', '53', ' ', 'los', ' ', 'nuevos c', 'asos ', 'confirmados ', 'de COVID', '-', ' ', '19 ', 'en la región. ', 'Estos nuevos casos sumados a los confirmados anteriormente, nos entregan la ', 'siguiente distribución comunal:', ' ', 'Comuna de ', 'residencia', ' ', 'N° de casos', ' ', ' ', ' ', 'Casos nuevos', ' ', ' ', ' ', 'Casos activos', ' ', 'Angol', ' ', '2', '59', ' ', '11', ' ', '84', ' ', 'Carahue', ' ', '68', ' ', '-', ' ', '5', ' ', 'Chol', '-', 'Chol', ' ', '24', ' ', '-', ' ', '6', ' ', 'Collipulli', ' ', '37', ' ', '-', ' ', '7', ' ', 'Cunco', ' ', '4', '8', ' ', '1', ' ', '16', ' ', 'Curacautín', ' ', '1', '8', ' ', '7', ' ', '9', ' ', 'Curarrehue', ' ', '4', ' ', '-'

ValueError: 'Total General' is not in list

In [25]:
if len(reportes_diarios)>0:
    for r in reportes_diarios:
        texto     = utils_araucania.get_info(r['fecha'], r['url'])       # Obtenemos el texto de la primera pagina del informe
        data      = utils_araucania.get_data(texto, r['fecha'])          # Obtenemos la data
        araucania.loc[len(araucania)] = data             # Creamos el data frame
else:
    print ("Todo Ok..")

Obteniendo información del: 12 de Junio
Obteniendo información del: 13 de Junio
Obteniendo información del: 14 de Junio
Obteniendo información del: 15 de Junio


ValueError: 'Total General' is not in list